In [1]:
import pandas as pd
import itertools
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import collections
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st

In [15]:
data = pd.read_csv('./data/classifiers.csv')
data.head(100)

,Номер выборки,a1,a2,a3,a4
0,1,86,50,93,13
1,2,85,74,55,35
2,3,53,92,58,51
3,4,44,41,56,37
4,5,2,18,99,26
5,6,5,68,35,17


Будем сравнивать выборки, опираясь на **U-критерий Манна-Уитни-Улкоксона**, так как можно считать выборки независимыми. 
- *Постановка задачи*: имеет место следующая альтернатива:
$$
\begin{align*}
    H_0: & F_{X_1} (x) = F_{X_2} (x), \\
    H_1: & F_{X_1} (x) = F_{X_2} (x + \Delta), \qquad \Delta \neq 0.
\end{align*}
$$

- *Рассчет статистики*: рассматриваем вариационный ряд объединенной выборки:
$$
X_{(1)} \leq \ldots \leq X_{(n_1 + n_2)}, \text{ где } X = X_{1}^{n_1} \cup X_{2}^{n_2}.
$$
Тогда статистику считаем по формуле:
$$
R(X_{1}^{n_1}, X_{2}^{n_2}) = \sum \limits_{i = 1}^{n_1} \mathrm{rank} \, X_{1i}. 
$$

- *Нулевое распределение*: табличное, рассчет перебором ранговых комбинаций элементов $X_1^{n_1}$ в объединенном вариационном ряду. 

- *Поправка на множественную гипотезу*: по Бенджамини-Хохбергу $\alpha_j = \dfrac{\alpha \cdot j}{6}, \, j = 1, \ldots, m$. Здесь $\displaystyle m = 6$ - число наблюдений.

- *Условие непринятия*: отвергаем $H_0$ тогда и только тогда, когда значение статистики $\displaystyle r \notin \left[ R_{\frac{\alpha^*}{2}}, R_{\frac{1 - \alpha^*}{2}} \right]$.

Имплементация по Россу почему-то дает стабильную фигню, поэтому сделаем чуток иначе

In [3]:
import numpy as np

# Данная имплементация взята по книге Ross (2009)

def cum_sum_prob(N, M, K):

    if (N == 1) & (M == 0):
        if K <= 0:
            return 0
        else:
            return 1
    elif (N == 0) & (M == 1):
        if K < 0:
            return 0
        else:
            return 1

    else:
        if N == 0:
            return (M / (N + M)) * cum_sum_prob(N, M - 1, K)
        elif M == 0:
            return (N / (N + M)) * cum_sum_prob(N - 1, M, K - N - M)
        else:
            return (N / (N + M)) * cum_sum_prob(N - 1, M, K - N - M) + (M / (N + M)) * cum_sum_prob(N, M - 1, K)


def mann_whitney_p(n, m, t):
    # Двусторонняя альтернатива
    pvalue = 2 * min(cum_sum_prob(n, m, t), 1 - cum_sum_prob(n, m, t - 1))
    return pvalue


def mann_whitney(x, y):
    # Упорядоченный объединенный массив + предподсчет рангов
    union = np.array(x + y)
    ordered_union = union.argsort()
    ranks = ordered_union.argsort()

    # Изменение на 1 нумерации
    ranks = ranks + 1

    # Сумма рангов 
    n, m = len(x), len(y)
    t = np.sum(ranks[:n])
    T = t

    pvalue = mann_whitney_p(n, m, T)

    return T, pvalue

In [4]:
m = 6
alpha_0 = 0.05

sample_dict = {}
cnt = 0
for i in range(1, 4):
    for j in range(i + 1, 5):
        cnt += 1
        
        # Бенджамини-Хохберг 
        alpha = alpha_0 * cnt / m

        # Создаем словарик для удобной визуализации:
        ai, aj = 'a' + str(i), 'a' + str(j)
        print(ai, aj)

        t, pvalue = mann_whitney(data[aj].to_numpy(), data[ai].to_numpy())
        t_st, pvalue_st = st.mannwhitneyu(data[ai].to_numpy(), data[aj].to_numpy())
        print('Statval: {} and {}, pvalue: {} and {}'.format(t, t_st, pvalue, pvalue_st))

        sample_dict[ai+', '+aj] = pvalue
        sample_dict['mod_alpha' + str(i) + str(j)] = alpha

sample_dict

a1 a2
Statval: 21 and 15.0, pvalue: 0.0021645021645021645 and 0.6991341991341992
a1 a3
Statval: 21 and 10.0, pvalue: 0.0021645021645021645 and 0.24025974025974026
a1 a4
Statval: 21 and 23.0, pvalue: 0.0021645021645021645 and 0.48484848484848486
a2 a3
Statval: 21 and 14.0, pvalue: 0.0021645021645021645 and 0.5887445887445888
a2 a4
Statval: 21 and 30.0, pvalue: 0.0021645021645021645 and 0.06493506493506493
a3 a4
Statval: 21 and 33.5, pvalue: 0.0021645021645021645 and 0.01612241534330035


{'a1, a2': 0.0021645021645021645,
 'mod_alpha12': 0.008333333333333333,
 'a1, a3': 0.0021645021645021645,
 'mod_alpha13': 0.016666666666666666,
 'a1, a4': 0.0021645021645021645,
 'mod_alpha14': 0.025000000000000005,
 'a2, a3': 0.0021645021645021645,
 'mod_alpha23': 0.03333333333333333,
 'a2, a4': 0.0021645021645021645,
 'mod_alpha24': 0.041666666666666664,
 'a3, a4': 0.0021645021645021645,
 'mod_alpha34': 0.05000000000000001}

Функция подсчета ранга через тупой вариант (machine.learning) вторая имплементация:

In [21]:
def rank(x1, x2):
    union = np.concatenate((x1, x2))
    union.sort()
    res = 0
    for x_1 in x1:
        res += 1
        for x_12 in union:
            res += int(x_1 > x_12)
    return res

def get_pvalue(sum_cum_probs, rank): 
    pvalue = 2 * min(sum_cum_probs[rank], 1 - sum_cum_probs[rank])
    return pvalue